# Ngrok을 통한 vllm Endpoint 제공 방법

## 개요

* colab 내에서 vllm을 통해 LLM 모델 실행 및 엔드포인트 실행
* 해당 엔드포인트를 외부에서 접근가능하도록 ngrok을 통해 외부 endpoint와 연결

## * Google Drive 연결 및 마운트

In [2]:
# 1. 구글 드라이브 연결
from google.colab import drive
import os

print("📂 구글 드라이브 연결 중...")
drive.mount('/content/drive')

# 2. 작업 디렉토리 변경 (서버의 cd 명령와 동일)
# 실제 프로젝트 폴더 경로로 이동
PROJECT_PATH = '/content/drive/MyDrive/ngrok_api_server'

if not os.path.exists(PROJECT_PATH):
    # 폴더가 없으면 생성 (최초 1회)
    os.makedirs(PROJECT_PATH)
    print(f"🆕 폴더 생성됨: {PROJECT_PATH}")
    print("⚠ 구글 드라이브에 가서 필요한 파일(train.py, config.ini 등)을 넣어주세요!")
else:
    os.chdir(PROJECT_PATH)
    print(f"✅ 작업 경로 이동 완료: {os.getcwd()}")

# 3. 현재 파일 목록 확인 (제대로 이동했는지 체크)
print("\n[현재 폴더 파일 목록]")
print(os.listdir())


## * ngrok 및 vllm 패키지 설치

In [ ]:
!pip install vllm pyngrok

* 패키지 설치 후, 환경변수 설정 필요
  * NGROK_TOKEN
    * ngrok token을 키값으로 입력
    * https://dashboard.ngrok.com/get-started/your-authtoken

## * Ngrok Endpoint URL 생성

In [7]:
import os
from pyngrok import ngrok
from google.colab import userdata

NGROK_TOKEN = userdata.get('NGROK_TOKEN')
os.environ["NGROK_TOKEN"] = NGROK_TOKEN

ngrok.set_auth_token(NGROK_TOKEN)

public_url = ngrok.connect(8000).public_url
print(f"🚀 API 주소 생성 완료: {public_url}")
print(f"👉 이 주소를 복사해서 사용하세요: {public_url}/v1")

## * vllm 서버 실행

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model Qwen/Qwen3-4B-AWQ \
  --max-model-len 2048 \
  --gpu-memory-utilization 0.85 \
  --disable-log-stats \
  --enforce-eager \
  --max-num-seqs 1

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model Qwen/Qwen3-4B-AWQ \
  --max-model-len 2048 \
  --speculative-config '{"method": "ngram", "ngram_prompt_lookup_max": 4, "ngram_prompt_lookup_min": 1, "num_speculative_tokens": 5}' \
  --gpu-memory-utilization 0.85 \
  --max-model-len 2048 \
  --max-num-seqs 1 \
  --disable-log-stats \
  --enforce-eager

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model Eslzzyl/Qwen3-4B-Instruct-2507-AWQ \
  --max-model-len 2048 \
  --speculative-config '{"method": "eagle", "model": "AngelSlim/Qwen3-4B_eagle3", "num_speculative_tokens": 5}' \
  --gpu-memory-utilization 0.85 \
  --max-model-len 2048 \
  --max-num-seqs 1 \
  --disable-log-stats \
  --enforce-eager